# **SpaceX  Falcon 9 First Stage Landing Prediction**


## Exploring and Preparing Data

## Objectives

In this lab, we will perform Exploratory Data Analysis and Feature Engineering using `Pandas` and `Matplotlib`:

*   Exploratory Data Analysis
*   Preparing Data  Feature Engineering

First, we import the following libraries into the lab:

In [ ]:
# Importing required libraries
import pandas as pd # software library written for the Python programming language for data manipulation and analysis
import numpy as np # library adding support to multi-dimensional arrays, matrices and functions to operate on these arrays
import matplotlib.pyplot as plt # library for python and pyplot gives us a MatLab like plotting framework
import seaborn as sns # visualization library based on matplotlib, providing a high-level interface for drawing attractive and informative statistical graphics

## Exploratory Data Analysis


First, we read the SpaceX dataset into a Pandas dataframe and print its summary.

In [ ]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_2.csv"

# Importing the dataset from the above url
df = pd.read_csv(url)
df.head(5)

We try to see how the `FlightNumber` (indicating the continuous launch attempts) and `Payload` variables would affect the launch outcome.

We can plot out the <code>FlightNumber</code> vs <code>PayloadMass</code>and overlay the outcome of the launch. We see that as the flight number increases, the first stage is more likely to land successfully. The payload mass also appears to be a factor, even with more massive payloads, the first stage often returns successfully.

In [ ]:
sns.catplot(y="PayloadMass", x="FlightNumber", hue="Class", data=df, aspect = 5)
plt.xlabel("Flight Number", fontsize=20)
plt.ylabel("Pay load Mass (kg)", fontsize=20)
plt.show()

Next, we will drill down to each site to visualize its detailed launch records.

### Visualization of the relationship between Flight Number and Launch Site

We use the function <code>catplot</code> to plot <code>FlightNumber</code> vs <code>LaunchSite</code>, set the  parameter <code>x</code> parameter to <code>FlightNumber</code>, set the <code>y</code> to <code>Launch Site</code> and set the parameter <code>hue</code> to <code>'class'</code>.

In [ ]:
# Plot a scatter point chart with x axis to be Flight Number and y axis to be the launch site, and hue to be the class value
sns.catplot(y="LaunchSite", x="FlightNumber", hue="Class", data=df, aspect=2, jitter=False)
plt.xlabel("Flight Number", fontsize=20)
plt.ylabel("Launch Site", fontsize=20)
plt.show()

### Visualization of the relationship between Payload Mass and Launch Site

We also want to observe if there is any relationship between launch sites and their payload mass.

In [ ]:
# Plotting a scatter point chart with x axis to be Pay Load Mass (kg) and y axis to be the launch site, and hue to be the class value
sns.catplot(y="LaunchSite", x="PayloadMass", hue="Class", data=df, aspect=2, jitter=False)
plt.xlabel("Pay load Mass (kg)", fontsize=20)
plt.ylabel("Launch Site", fontsize=20)
plt.show()

Now, we observe Payload Mass vs Launch Site scatter point chart and we find that for the <code>VAFB-SLC</code> launch site there are no rockets launched for heavy payload mass (greater than 10000).

### Visualization of the relationship between success rate of each orbit type

Next, we want to visually check if there are any relationship between success rate and orbit type.

So, we create a `bar chart` for the success rate of each orbit.

In [ ]:
# Utilization of groupby() method on Orbit column and get the mean of Class column
temp = df.groupby('Orbit')['Class'].mean().reset_index()
sns.barplot(y="Class", x="Orbit", data=temp)

plt.xlabel("Orbit", fontsize=20)
plt.ylabel("Success Rate", fontsize=20)
plt.show()

Based on the plotted bar chart, we identify that <code>ES-L1</code>, <code>GEO</code>, <code>HEO</code> and <code>SSO</code> orbits have the highest success rates.

### Visualization of the relationship between FlightNumber and Orbit type

For each orbit, we want to see if there is any relationship between FlightNumber and Orbit type.

In [ ]:
# Plotting a scatter point chart with x axis to be FlightNumber and y axis to be the Orbit, and hue to be the class value
sns.catplot(y="Orbit", x="FlightNumber", hue="Class", data=df, aspect=2, jitter=False)
plt.xlabel("Flight Number", fontsize=20)
plt.ylabel("Orbit", fontsize=20)
plt.show()

We can observe that in the <code>LEO</code> orbit, success seems to be related to the number of flights. Conversely, in the <code>GTO</code> orbit, there appears to be no relationship between flight number and success.

### Visualization of the relationship between Payload Mass and Orbit type

Similarly, we can plot the Payload Mass vs Orbit scatter point charts to reveal the relationship between Payload Mass and Orbit type.

In [ ]:
# Plotting a scatter point chart with x axis to be Payload Mass and y axis to be the Orbit, and hue to be the class value
sns.catplot(y="Orbit", x="PayloadMass", hue="Class", data=df, aspect=2, jitter=False)
plt.xlabel("Pay load Mass (kg)", fontsize=20)
plt.ylabel("Orbit", fontsize=20)
plt.show()

With heavy payloads the successful landing or positive landing rate are more for <code>Polar</code>, <code>LEO</code> and <code>ISS</code>.

However, for <code>GTO</code>, it's difficult to distinguish between successful and unsuccessful landings as both outcomes are present.

### Visualization of the launch success yearly trend

We can plot a line chart with x axis to be <code>Year</code> and y axis to be average success rate, to get the average launch success trend.

The function that will help us get the year from the date is the following:

In [ ]:
year = []

def Extract_year():
    for i in df["Date"]:
        year.append(i.split("-")[0])
    return year

Extract_year()

df['Date'] = year
df.head()

In [ ]:
# Plotting a line chart with x axis to be the extracted year and y axis to be the success rate
plt.figure(figsize=(10, 6))
sns.lineplot(data=df, x="Date", y="Class", ci=None)
plt.xlabel("Date", fontsize=20)
plt.ylabel("Success Rate", fontsize=20)
plt.title('Average yearly launch success rate')
plt.show()

We can observe that the success rate since 2013 kept increasing till 2020.

## Features Engineering

We have obtained some preliminary insights about how each important variable would affect the success rate and we will select the features that will be used in success prediction in the future module.

In [ ]:
features = df[['FlightNumber', 'PayloadMass', 'Orbit', 'LaunchSite', 'Flights', 'GridFins', 'Reused', 'Legs', 'LandingPad', 'Block', 'ReusedCount', 'Serial']]
features.head()

### Creation of dummy variables to categorical columns

We use the function <code>get_dummies</code> and <code>features</code> dataframe to apply OneHotEncoder to the column <code>Orbits</code>, <code>LaunchSite</code>, <code>LandingPad</code>, and <code>Serial</code>. We assign the value to the variable <code>features_one_hot</code> and display the results using the method head. The result dataframe must include all features including the encoded ones.

In [ ]:
# Utilization of get_dummies() function on the categorical columns
features_one_hot = pd.get_dummies(features, columns=['Orbit', 'LaunchSite', 'LandingPad', 'Serial'])
features_one_hot.head()

### Casting of all numeric columns to `float64`

Now that the <code>features_one_hot</code> dataframe only contains numbers, we will cast the entire dataframe to variable type <code>float64</code>.

In [ ]:
# Utilization of astype() function
features_one_hot.astype(float)

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent, in the next lab we will provide data in a pre-selected date range.

features_one_hot.to_csv('dataset_part_3.csv', index=False)

## Exploratory Data Analysis (EDA) using SQL

Execute SQL queries to perform EDA at more depth.

### Connection to the database

First, we have to load the SQL extension and establish a connection with the database.

In [ ]:
!pip install ipython-sql
!pip install ipython-sql prettytable

In [ ]:
%load_ext sql

In [ ]:
import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect("data_EDA_SQL.db")
cur = con.cursor()

In [ ]:
%sql sqlite:///data_EDA_SQL.db

**Note: The below code is added to remove blank rows from the table.**

In [ ]:
# Dropping the table if it already exists
%sql DROP TABLE IF EXISTS SPACEXTABLE;

In [ ]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

Displaying the names of the unique launch sites in the space mission.

In [ ]:
%sql select distinct(LAUNCH_SITE) from SPACEXTBL

Displaying 5 records where launch sites begin with the string 'CCA'. 

In [ ]:
%sql select * from SPACEXTBL where LAUNCH_SITE like 'CCA%' limit 5

Displaying the total payload mass carried by boosters launched by NASA (CRS).

In [ ]:
%sql select sum(PAYLOAD_MASS__KG_) as 'Total payload mass by NASA (CRS)' from SPACEXTBL where Customer = 'NASA (CRS)'

Displaying average payload mass carried by booster version F9 v1.1.

In [ ]:
%sql select avG(PAYLOAD_MASS__KG_) as 'Average payload mass by F9 v1.1' from SPACEXTBL where Booster_Version = 'F9 v1.1'

Listing the date when the first successful landing outcome in ground pad was achieved.

In [ ]:
%sql select distinct(Landing_Outcome) from SPACEXTBL

In [ ]:
%sql select min(Date) as 'Date of first successful landing (ground pad)' from SPACEXTBL where Landing_Outcome = 'Success (ground pad)'

Listing the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000.

In [ ]:
%sql select Booster_Version from SPACEXTBL where Landing_Outcome = 'Success (drone ship)' and PAYLOAD_MASS__KG_ > 4000 and PAYLOAD_MASS__KG_ < 6000

Listing the total number of successful and failure mission outcomes.

In [ ]:
%sql select distinct(Mission_Outcome) from SPACEXTBL

In [ ]:
%sql select count(Mission_Outcome) from SPACEXTBL where Mission_Outcome = 'Success' or Mission_Outcome = 'Failure (in flight)'

Listing the names of the booster versions which have carried the maximum payload mass.

In [ ]:
%sql select Booster_Version from SPACEXTBL where PAYLOAD_MASS__KG_ = (select max(PAYLOAD_MASS__KG_) from SPACEXTBL)

Listing the records which will display the month names, failure landing_outcomes in drone ship, booster versions, launch sites for the months in year 2015.

In [ ]:
%sql select Date, substr(Date, 6, 2) as Month, substr(Date, 0, 5) as Year, Landing_Outcome, Booster_Version, Launch_Site from SPACEXTBL where substr(Date, 0, 5) = '2015' and Landing_Outcome = 'Failure (drone ship)'

Ranking the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.

In [ ]:
%sql select Landing_Outcome, count(*) as 'landing_count' from SPACEXTBL where (Date between '2010-06-04' and '2017-03-20') group by Landing_Outcome order by count(*) desc